In [1]:
#imports 

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [4]:
#Tensorflow model zoo downloader
def load_model(model_name):    
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))

    return model

In [5]:
## In the Detection model file, there is a labelmap.pbtxt made for this project. Set the path to run this block
PATH_TO_LABELS = r'C:\Users\LG\Documents\models\research\object_detection\Images\labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [6]:
## A path for your images to be tested from should be set here.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(r'C:\Users\LG\Desktop')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('C:/Users/LG/Desktop/images.jpg')]

In [ ]:
#This is the pre-trained object_detection model for this project (Currrent loss:0.253)
#The model is located in the Detection model file (select the path to saved_model.pb file)
detection_model1 = tf.saved_model.load(r'C:\Users\LG\Documents\models\research\object_detection\inference_graph1\saved_model')

In [78]:
#This function performs detection on the image passed.
def run_inference_for_single_image(model, image):
    
    image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]
    
  # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
    if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict


In [115]:
#In the Classification models file are the pre-trained models. set the path to run this block.
import pickle
import bz2
import _pickle as cPickle

# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data
model  = decompress_pickle('orange.pbz2')
model0 = decompress_pickle('banana.pbz2')
model1 = decompress_pickle('pineapple.pbz2')

In [116]:
#This class is to set the properties for each of the detection boxes. the more the objects, the more the detection boxes can be found on image
class Boxes():
    
    def __init__(self, i, image):
        Categories = ['Ripe:', 'Unripe:']
        self.image = image
        shape = list(self.image.shape)
        im_height = int(shape[0])
        im_width =int(shape[1])
        output_dict = run_inference_for_single_image(detection_model1, self.image)
        self.ymin = int(output_dict['detection_boxes'][i][0]*im_height)
        self.xmin = int(output_dict['detection_boxes'][i][1]*im_width)
        self.ymax = int(output_dict['detection_boxes'][i][2]*im_height)
        self.xmax = int(output_dict['detection_boxes'][i][3]*im_width)
        self.roi = self.image[self.ymin:self.ymax,self.xmin:self.xmax]
        self.label = str(output_dict['detection_classes'][i])
        self.ripeness = '9'
        self.colour = 3
        
#         if  (self.label == '1') or(self.label == '2') or (self.label == '4') :
#             pass
        #Orange
        if (self.label == '3'):
            sample = cv2.resize(self.roi, (100, 100))
            sample = sample.flatten()
            sample = np.array(sample).reshape(1,-1)
            self.label = Categories[model.predict(sample)[0]]

            if self.label == 'Ripe:':
                
                self.colour = (0,165,255)
                self.ripeness = str(model.predict_proba(sample)[0][0]*100)[:2]+ '%'
            else:
                self.ripeness = str(model.predict_proba(sample)[0][1]*100)[:2]+ '%'
                self.colour = (32,80,1)
#         else:
#              pass
#         #Banana
        if  (self.label == '1') or(self.label == '2') or (self.label == '4'):
            sample = cv2.resize(self.roi, (100, 100))
            sample = sample.flatten()
            sample = np.array(sample).reshape(1,-1)
            self.label = Categories[model0.predict(sample)[0]]


            if self.label == 'Ripe:':
                
                self.colour = (0,255,255)
                self.ripeness = str(model0.predict_proba(sample)[0][0]*100)[:2]+ '%'
            else:
                self.ripeness = str(model0.predict_proba(sample)[0][1]*100)[:2]+ '%'
                self.colour = (0,255,0)
#          Pineapple
        if (self.label == '5'):
            sample = cv2.resize(self.roi, (100, 100))
            sample = sample.flatten()
            sample = np.array(sample).reshape(1,-1)
            self.label = Categories[model1.predict(sample)[0]]

            if self.label == 'Ripe:':
                
                self.colour = (0,0,255)
                self.ripeness = str(model1.predict_proba(sample)[0][0]*100)[:2]+ '%'
            else:
                self.ripeness = str(model1.predict_proba(sample)[0][1]*100)[:2]+ '%'
                self.colour = (255,0,0)

        

In [26]:
#Live Detection (WEBCAM NEEDED)
import cv2
cap = cv2.VideoCapture(0)
Categories = ['Ripe:', 'Unripe:']


while True:
    ret, image_np = cap.read()
    # Actual detection.
    output_dict= run_inference_for_single_image(detection_model1, image_np)
    
    #Detection_Boxes
    box0 = Boxes(0, image_np)
    box1 = Boxes(1, image_np)
    box2 = Boxes(2, image_np)
    
    try:
        if output_dict['detection_scores'][0]>= 0.5:

            cv2.rectangle(image_np,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
            cv2.rectangle(image_np,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
            cv2.putText(image_np, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(image_np, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

        if output_dict['detection_scores'][1]>= 0.5:

            cv2.rectangle(image_np,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
            cv2.rectangle(image_np,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
            cv2.putText(image_np, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(image_np, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)


        if output_dict['detection_scores'][2]>= 0.5:

            cv2.rectangle(image_np,(box2.xmin,box2.ymin),(box2.xmax,box2.ymax),box2.colour,2)
            cv2.rectangle(image_np,(box2.xmin,box2.ymin-60),(box2.xmax,box2.ymin),box2.colour,-1)
            cv2.putText(image_np, box2.label, (box2.xmin, box2.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(image_np, box2.ripeness, (box2.xmin, box2.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    except:
        pass

    cv2.imshow('object detection', cv2.resize(image_np, (800,600),-1))
    if cv2.waitKey(1) & 0xFF == 27:
        cap.release()
        cv2.destroyAllWindows()
        break

In [194]:
#Image Detection (set path to image, or pass name of image if image is in this directory)
import cv2
import numpy as np
# image = "download(1).jpg"
image = cv2.imread("images.jpg")
# image = cv2.resize(image, (800, 600))
# shape = list(image.shape)
# im_height = int(shape[0])
# im_width =int(shape[1])
    

def RBF(detection_model, image):
    
    output_dict = run_inference_for_single_image(detection_model, image)
    box0 = Boxes(0, image)
    box1 = Boxes(1, image)
    box2 = Boxes(2, image)
    if (output_dict['detection_classes'][0]==3) & (output_dict['detection_scores'][0]>= 0.5):

        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    if (output_dict['detection_classes'][1]==3) & (output_dict['detection_scores'][1]>= 0.5):

        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    #pineaple

    if (output_dict['detection_classes'][0]==5) & (output_dict['detection_scores'][0]>= 0.5):
        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    if (output_dict['detection_classes'][1]==5) & (output_dict['detection_scores'][1]>= 0.5):
        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    #banana
#     if (output_dict['detection_classes'][1]==4) & (output_dict['detection_scores'][1]>= 0.5):
    if (output_dict['detection_classes'][0]==2) & (output_dict['detection_scores'][0]>= 0.5):
        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    if (output_dict['detection_classes'][1]==2) & (output_dict['detection_scores'][1]>= 0.7):
        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
#     image = cv2.resize(image, (800, 600))
    return image
    

In [195]:
#Display
cv2.imshow('image',RBF(detection_model1, image))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
#Video Detection for processing and filming
import cv2
cap = cv2.VideoCapture('Unripe.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('unrapeban.mp4',cv2.VideoWriter_fourcc('X','V','I','D'), 30, (frame_width,frame_height))
Categories = ['Ripe:', 'Unripe:']
while True:
    ret, image = cap.read()
    # Actual detection.
    try:
        output_dict = run_inference_for_single_image(detection_model1, image)
        # I set to best three boxes to be detected for this project. more can be created 
        box0 = Boxes(0, image)
        box1 = Boxes(1, image)
        box2 = Boxes(2, image)
    except:
        pass
# first Box condition for three classes 
    if (output_dict['detection_classes'][0]==5) & (output_dict['detection_scores'][0]>= 0.4):

        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
              
    if  (output_dict['detection_classes'][0]==3) & (output_dict['detection_scores'][0]>= 0.1):

        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    if (output_dict['detection_classes'][0]==1) or (output_dict['detection_classes'][0]==2) & (output_dict['detection_scores'][0]>= 0.1):
        image = cv2.rectangle(image,(box0.xmin,box0.ymin),(box0.xmax,box0.ymax),box0.colour,2)
        image = cv2.rectangle(image,(box0.xmin,box0.ymin-60),(box0.xmax,box0.ymin),box0.colour,-1)
        image = cv2.putText(image, box0.label, (box0.xmin, box0.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box0.ripeness, (box0.xmin, box0.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    
    
# second Box condition for three classes     
        
    if  (output_dict['detection_classes'][1]==5) & (output_dict['detection_scores'][1]>= 0.4):
        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    if  (output_dict['detection_classes'][1]==3) & (output_dict['detection_scores'][1]>= 0.1):
        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    if (output_dict['detection_classes'][1]==1) or (output_dict['detection_classes'][1]==2) & (output_dict['detection_scores'][1]>= 0.9):
        image = cv2.rectangle(image,(box1.xmin,box1.ymin),(box1.xmax,box1.ymax),box1.colour,2)
        image = cv2.rectangle(image,(box1.xmin,box1.ymin-60),(box1.xmax,box1.ymin),box1.colour,-1)
        image = cv2.putText(image, box1.label, (box1.xmin, box1.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box1.ripeness, (box1.xmin, box1.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    
    
#Third Box condition for three classes    
    
    if  (output_dict['detection_classes'][2]==5) & (output_dict['detection_scores'][2]>= 0.5):

        image = cv2.rectangle(image,(box2.xmin,box2.ymin),(box2.xmax,box2.ymax),box2.colour,2)
        image = cv2.rectangle(image,(box2.xmin,box2.ymin-60),(box2.xmax,box2.ymin),box2.colour,-1)
        image = cv2.putText(image, box2.label, (box2.xmin, box2.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box2.ripeness, (box2.xmin, box2.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    if  (output_dict['detection_classes'][2]==3) & (output_dict['detection_scores'][2]>= 0.1):
        image = cv2.rectangle(image,(box2.xmin,box2.ymin),(box2.xmax,box2.ymax),box2.colour,2)
        image = cv2.rectangle(image,(box2.xmin,box2.ymin-60),(box2.xmax,box2.ymin),box2.colour,-1)
        image = cv2.putText(image, box2.label, (box2.xmin, box2.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box2.ripeness, (box2.xmin, box2.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)        
        
    if (output_dict['detection_classes'][2]==1) or (output_dict['detection_classes'][2]==2) & (output_dict['detection_scores'][1]>= 0.9):
        image = cv2.rectangle(image,(box2.xmin,box2.ymin),(box2.xmax,box2.ymax),box2.colour,2)
        image = cv2.rectangle(image,(box2.xmin,box2.ymin-60),(box2.xmax,box2.ymin),box2.colour,-1)
        image = cv2.putText(image, box2.label, (box2.xmin, box2.ymin-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        image = cv2.putText(image, box2.ripeness, (box2.xmin, box2.ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    
    
    #saving the processed image frames
    if (ret == True):
        out.write(image)
        cv2.imshow('object detection', cv2.resize(image, (800,600),-1))
        if cv2.waitKey(1) & 0xFF == 27:
            cap.release()
            out.release()
            cv2.destroyAllWindows()
    else:
        break
        cap.release()
        cv2.destroyAllWindows()
        out.release()
#         break
out.release()

In [43]:
# Run to be able to view processed video
out.release()